In [142]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

In [143]:
from sklearn.linear_model import LogisticRegression

In [144]:
from sklearn.preprocessing import LabelBinarizer

In [145]:
data_history=pd.read_csv('Historical-transaction-data.csv')
data_shops=pd.read_csv('Store-info.csv')

In [146]:
number_of_days=data_history.value_counts('transaction_date').count()
list_shop=[]
count=0
for shop_id in data_shops['shop_id']:
    if count<=99:
        list_shop.append(shop_id)
    else:
        break
    count=count+1
shop_array=np.array(list_shop)

In [147]:
shops=[]
x=[]
y=[]


In [148]:
class shop:
    df=''
    shop_id=''
    full_income=0
    mean_income=0
    min_income=0
    max_income=0
    median_income=0
    mode_income=0
    count_coustomers=0
    count_coustomer_sequence=0
    count_fixed_coustomers=0
    max_items_per_time=0
    min_items_per_time=0
    mean_items_per_time=0
    median_items_per_time=0
    mode_items_per_time=0
    count_sells=0
    mean_count_sold_types=0
    max__count_sold_types=0
    min__count_sold_types=0
    mode__count_sold_types=0
    median_count_sold_types=0
    area=0
    points=0
    X=[]
    def __init__(self,shop_id,df,area):
        self.shop_id=shop_id
        self.area=area
        self.df=df
        self.income_info()
        self.item_info()
        self.costomers_items_info()
        self.customer_info()
        #min_income,max_income,median_income,mode_income,,max_count_coustomers,min_count_coustomers,mode_count_coustomers,
        #median_count_coustomers,max_items_per_time,min_items_per_time,
        #,median_items_per_time,mode_items_per_time,count_sells,
        #,max__count_sold_types,min__count_sold_types,mode__count_sold_types,median_count_sold_types
        
        #self.median_count_sold_types,self.median_items_per_time,self.mode_items_per_time,self.max_items_per_time,self.min_items_per_time,self.min_income,self.max_income,self.median_income,self.mode_income,
        self.X=[self.max_items_per_time,self.max_income,self.full_income,self.mean_income,self.count_coustomers,self.count_coustomer_sequence,self.count_fixed_coustomers,self.mean_count_sold_types,self.area,self.count_sells]
        
    def income_info(self):
        incomes=np.array(self.df['item_price'])
        for inc in incomes:
            self.full_income=self.full_income+inc
        summery=np.array(self.df.drop(['invoice_id','quantity_sold'],axis=1).describe())
        self.max_income=summery[7][0]
        self.min_income=summery[3][0]
        self.mean_income=summery[1][0]
        self.median_income=summery[5][0]
        self.mode_income=self.df.value_counts('item_price').keys()[0]
    def item_info(self):
        items=np.array(self.df['quantity_sold'])
        self.count_sells=items.sum()
        summery_2=self.df['item_description'].value_counts().describe()
        self.mean_count_sold_types=summery_2[1]
        self.max_count_sold_types=summery_2[7]
        self.min_count_sold_types=summery_2[3]
        self.median_count_sold_types=summery_2[5]
        self.mode_count_sold_types=np.array(self.df['item_description'].value_counts())[0]
        
    def costomers_items_info(self):
        summery=np.array(self.df.value_counts(['customer_id','invoice_id']))
        self.mean_items_per_time=summery.mean()
        self.max_items_per_time=max(summery)
        self.min_items_per_time=min(summery)
        self.mode_items_per_time=np.array(pd.Series(summery).value_counts())[0]
        self.median_items_per_time=np.array(pd.Series(summery).describe())[5]
    def customer_info(self):
        summery=np.array(self.df.value_counts(['customer_id']))
        self.count_coustomers=len(summery)
        custom_invoice_count_df=pd.DataFrame(self.df.value_counts(['customer_id','invoice_id']))
        self.count_coustomer_sequence=custom_invoice_count_df.shape[0]
        summery_2=np.array(custom_invoice_count_df.value_counts('customer_id'))
        count_fixed=0
        for i in summery_2:
            if i>=number_of_days/14:
                count_fixed=count_fixed+1
        self.count_fixed_coustomers=count_fixed       

In [149]:
def data_separation_for_shops():
    index=0
    profiles=np.array(data_shops['shop_profile'])
    for shop_id in list_shop:
        shop_df=data_history.loc[data_history['shop_id']==shop_id]
        current_shop=shop(shop_id,shop_df,data_shops['shop_area_sq_ft'][index])
        shops.append(current_shop)
        x.append(np.array(current_shop.X))
        y.append(profiles[index])
        index=index+1

In [150]:
data_separation_for_shops()   #create shops

In [151]:
col=[]
print(len(y),len(x))

In [152]:
for i in range(0,len(x)):
    for j in range(0,2):
        if i==0:
            col.append([])
        col[j].append(x[i][j]) 

In [153]:
for c in col:
    print(len(c))

In [154]:
#points
def rank(array):
    marks=[]
    for item in array:
        mark=0
        for i in array:
            if i<item:
                mark=mark+1
        marks.append(mark)
    return np.array(marks)                

In [155]:
i=0
total=0
for c in col:
    marks=rank(c)
    if i==8:
        marks=marks*3
    if i==0:
        total=marks
    else:
        total=total+marks
    i=i+1
Z_score=total
     

In [156]:
Z_score

In [157]:
y_=[]
for i in y:
    if i=='High':
        y_.append(3)
    elif i=='Low':
        y_.append(1)
    else:
        y_.append(2)
trainX,testX,trainY,testY=train_test_split(Z_score,y_,random_state=1,test_size=0.3)

In [158]:
model=LogisticRegression()
model.fit(np.array(trainX).reshape(-1, 1),trainY)

In [159]:
model.predict(np.array(testX).reshape(-1,1))

In [160]:
#F1
from sklearn.metrics import classification_report
report=pd.DataFrame(classification_report(testY,model.predict(np.array(testX).reshape(-1,1)),output_dict=True))

In [161]:
report

In [162]:
from sklearn.metrics import confusion_matrix
m=confusion_matrix(testY,model.predict(np.array(testX).reshape(-1,1)))

In [163]:
m

In [164]:
pre1=m[0][0]/(m[0][0]+m[0][1]+m[0][2])
pre2=m[1][0]/(m[1][0]+m[1][1]+m[1][2])
pre3=m[2][0]/(m[2][0]+m[2][1]+m[2][2])
re1=m[0][0]/(m[0][0]+m[1][0]+m[2][0])
re2=m[0][1]/(m[0][1]+m[1][1]+m[2][1])
re3=m[0][2]/(m[0][2]+m[1][2]+m[2][2])
f11=2*pre1*re1/(pre1+re1)
f12=2*pre2*re2/(pre2+re2)
f13=2*pre3*re3/(pre3+re3)

In [165]:
f11

In [166]:
f12

In [167]:
f13

In [168]:
f=(f11+f12+f13)/3

In [169]:
f

In [170]:
list_shop=[]
ids=np.array(data_shops['shop_id'])
for i in range(100,124):
    list_shop.append(ids[i])
    
shops=[]
x=[]
def new_():
    index=100
    for shop_id in list_shop:
        shop_df=data_history.loc[data_history['shop_id']==shop_id]
        current_shop=shop(shop_id,shop_df,data_shops['shop_area_sq_ft'][index])
        shops.append(current_shop)
        x.append(np.array(current_shop.X))
        index=index+1
new_()

col=[]
print(len(x))

In [171]:
for i in range(0,len(x)):
    for j in range(0,10):
        if i==0:
            col.append([])
        col[j].append(x[i][j]) 
        
i=0
total=0
for c in col:
    marks=rank(c)
    if i==8:
        marks=marks*3
    if i==0:
        total=marks
    else:
        total=total+marks
    i=i+1
Z_score=total

In [172]:
Z_score

In [173]:
model.predict(Z_score.reshape(-1,1))